In [ ]:
results_dict1_2 = {
    {
        md:{n_warmups:{n_chains:{} for n_chains in N_CHAINS} for n_warmups in N_WARMUPS} for md in MAX_DOUBLINGS
    }
}

In [ ]:
# for sanity compare with the other way we do multi-chain sampling, should give same timing if I understand vectorization correctly
print('md, n_warmups, n_chains')

# no way to jit over the n_warmup or max_num_doublings
for md in MAX_DOUBLINGS:
    for n_warmups in N_WARMUPS:
        state = results_dict['warmup'][md][n_warmups]['init_states']
        tuned_params = results_dict['warmup'][md][n_warmups]['tuned_params']

        logdensity = partial(_logprob_fn, data=data_gpu)
        kernel = blackjax.nuts(logdensity, **tuned_params).step
        for n_chains in N_CHAINS:
            print(md, n_warmups, n_chains)
            init_states = [state for ii in range(n_chains)] # CHECK
            keys = random.split(rng_key, n_chains)
            _run_inference = jax.jit(partial(inference_loop_multiple_chains, 
                                             n_samples = 1000 // n_chains, # fair runtime comparison
                                             n_chains=n_chains, 
                                             kernel=kernel, 
                                            ))

            # compilation time
            t1 = time.time()
            _ = _run_inference(keys, init_states)
            t2 = time.time()
            results_dict1_2['inference'][md][n_warmups][n_chains]['comp_time'] = t2 - t1 

            # run time
            t1 = time.time()
            states, info = _run_inference(keys, init_states)
            t2 = time.time()
            results_dict1_2['inference'][md][n_warmups][n_chains]['run_time'] = t2 - t1
            
            # save states and info for future reference
            results_dict1_2['inference'][md][n_warmups][n_chains]['states'] = states
            results_dict1_2['inference'][md][n_warmups][n_chains]['info'] = info
            
            
print()
print('DONE!')

